# 1 Azure Maps

https://azure.microsoft.com/en-us/products/azure-maps/?msockid=2e39c66c693c66a5151fd200687567d0

https://learn.microsoft.com/en-us/azure/azure-maps/

### 🔍 Search
Create apps that display nearby points of interest (POIs), show locations on a map and perform geocoding to convert addresses into coordinates —or reverse geocoding to get addresses from coordinates.

### 🚦 Traffic
Build mobility solutions that reduce travel time, suggest alternate routes around traffic jams, provide insights into delays, and improve commuting during rush hour.

### 🗺️ Maps
Integrate clear, user-friendly maps into your cross-platform apps using the Web Map SDK or Render API. Azure Maps updates dynamically, ensuring users always get fresh, accurate information. Develop for web and mobile with dedicated SDKs.

### 🧭 Routing
Offer the shortest or fastest routes to multiple destinations. Support specialized routing for pedestrians and commercial vehicles, and enhance planning with isochrones, matrix routing, and batch routing.

### 🗃️ Data Registry Service
Store and manage organizational data assets securely in an Azure Storage Account linked to your Azure Maps account. Ensure data accuracy and reliability for streamlined operations and deeper insights.

### 🌦️ Weather Service
Powered by Accuweather, access current conditions, forecasts, and route-specific weather to enable smarter, weather-aware decisions in your apps.

### 🌍 Time Zones
Help users easily find the current time anywhere in the world. Get time zone info, UTC offsets, and daylight saving updates for any location.

### 🌐 Geolocation
Identify the country of an IP address to personalize content and services based on user location. Gain insights into geographic distribution and user demographics.

In [1]:
import datetime
import folium
import json
import os
import requests
import sys

from datetime import timedelta
from dotenv import load_dotenv
from IPython.display import IFrame, FileLink
from typing import Dict, List, Tuple, Optional

# Settings

In [2]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

In [3]:
print(f"Today is {datetime.datetime.today().strftime('%d-%b-%Y %H:%M:%S')}")

Today is 02-Sep-2025 08:40:10


In [4]:
!ls azure.env -lh

-rwxrwxrwx 1 root root 104 Sep  1 12:02 azure.env


In [5]:
load_dotenv("azure.env")

True

In [6]:
RESULTS_DIR = "results"

os.makedirs(RESULTS_DIR, exist_ok=True)

## Helper

In [7]:
class AzureMapsClient:
    """
    Azure Maps API Client for Python
    """
    def __init__(self, subscription_key: str):
        """
        Initialize Azure Maps client
        
        Args:
            subscription_key: Your Azure Maps subscription key
        """
        self.subscription_key = subscription_key
        self.base_url = "https://atlas.microsoft.com"

    def _make_request(self, endpoint: str, params: Dict) -> Dict:
        """
        Make authenticated request to Azure Maps API
        
        Args:
            endpoint: API endpoint
            params: Request parameters
            
        Returns:
            API response as dictionary
        """
        params['api-version'] = '1.0'
        params['subscription-key'] = self.subscription_key

        url = f"{self.base_url}/{endpoint}"

        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"API request failed: {e}")
            return {}

## Test

In [8]:
try:
    # Try to get key from environment variable
    subscription_key = os.getenv('AZURE_MAPS_KEY')

    if not subscription_key:
        # Prompt user for key if not found in environment
        subscription_key = input("Enter your Azure Maps subscription key: ")

    # Create client instance
    azure_maps = AzureMapsClient(subscription_key)
    print("✅ Azure Maps client initialized successfully!")

except Exception as e:
    print(f"❌ Setup failed: {e}")
    print("Please check your subscription key and internet connection")

✅ Azure Maps client initialized successfully!


## Example 1

In [9]:
address = 'Redmond, USA'

In [10]:
test_response = azure_maps._make_request('search/address/json', {'query': address, 'limit': 1})

if test_response and 'results' in test_response:
    print(f"OK. Test query returned {len(test_response['results'])} results")
else:
     print("ERROR. Connection test failed - please check your subscription key")

OK. Test query returned 1 results


In [11]:
test_response

{'summary': {'query': 'redmond',
  'queryType': 'NON_NEAR',
  'queryTime': 53,
  'numResults': 1,
  'offset': 0,
  'totalResults': 6,
  'fuzzyLevel': 1},
 'results': [{'type': 'Geography',
   'id': 'v44eSpinEz5drKY2oSsUsg',
   'score': 1,
   'entityType': 'Municipality',
   'matchConfidence': {'score': 1},
   'address': {'municipality': 'Redmond',
    'countrySecondarySubdivision': 'King',
    'countrySubdivision': 'WA',
    'countrySubdivisionName': 'Washington',
    'countrySubdivisionCode': 'WA',
    'countryCode': 'US',
    'country': 'United States',
    'countryCodeISO3': 'USA',
    'freeformAddress': 'Redmond, WA'},
   'position': {'lat': 47.674913, 'lon': -122.123995},
   'viewport': {'topLeftPoint': {'lat': 47.734, 'lon': -122.1649},
    'btmRightPoint': {'lat': 47.60034, 'lon': -121.95058}},
   'boundingBox': {'topLeftPoint': {'lat': 47.734002, 'lon': -122.1649},
    'btmRightPoint': {'lat': 47.600337, 'lon': -121.950576}},
   'dataSources': {'geometry': {'id': '00005557-4100

In [12]:
m = folium.Map(location=[test_response["results"][0]["position"]['lat'], test_response["results"][0]["position"]['lon']], zoom_start=12)
folium.Marker([test_response["results"][0]["position"]['lat'], test_response["results"][0]["position"]['lon']], popup=address).add_to(m)

output_file = os.path.join(RESULTS_DIR, "example1.html")
m.save(output_file)
print(f"✅ Map saved as {output_file}")

IFrame(src=output_file, width=1024, height=860)

✅ Map saved as results/example1.html


In [13]:
map_link = FileLink(path=output_file)
map_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/azure maps/results/example1.html

## Example 2

In [14]:
address = "39 Quai du Président Roosevelt, 92130 Issy-les-Moulineaux, France"

In [15]:
# Test connection with a simple request
test_response = azure_maps._make_request('search/address/json', {'query': address, 'limit': 1})

if test_response and 'results' in test_response:
    print(f"OK. Test query returned {len(test_response['results'])} results")
else:
     print("ERROR. Connection test failed - please check your subscription key")

OK. Test query returned 1 results


In [16]:
test_response

{'summary': {'query': '39 quai du président roosevelt 92130 issy les moulineaux france',
  'queryType': 'NON_NEAR',
  'queryTime': 138,
  'numResults': 1,
  'offset': 0,
  'totalResults': 1,
  'fuzzyLevel': 1},
 'results': [{'type': 'Point Address',
   'id': 'N8IS9PX5JarD5ZRJ7MXofA',
   'score': 0.9718004473256633,
   'matchConfidence': {'score': 0.9718004473256633},
   'address': {'streetNumber': '39',
    'streetName': 'Quai du Président Roosevelt',
    'municipality': 'Issy-les-Moulineaux',
    'neighbourhood': 'Bords et Val de Seine-Les Arches',
    'countrySecondarySubdivision': 'Hauts-de-Seine',
    'countrySubdivision': 'Île-de-France',
    'countrySubdivisionName': 'Île-de-France',
    'countrySubdivisionCode': 'IDF',
    'postalCode': '92130',
    'countryCode': 'FR',
    'country': 'France',
    'countryCodeISO3': 'FRA',
    'freeformAddress': '39 Quai du Président Roosevelt, 92130 Issy-les-Moulineaux',
    'localName': 'Issy-les-Moulineaux'},
   'position': {'lat': 48.834032

In [17]:
m = folium.Map(
    location=[test_response["results"][0]["position"]['lat'], test_response["results"][0]["position"]['lon']], 
    zoom_start=15,
    tiles='Esri.WorldImagery'
)

folium.TileLayer('OpenStreetMap').add_to(m)

folium.Marker(
    [test_response["results"][0]["position"]['lat'], test_response["results"][0]["position"]['lon']], 
    popup=address
).add_to(m)

output_file = os.path.join(RESULTS_DIR, "example2.html")
m.save(output_file)
print(f"✅ Map saved as {output_file}")

IFrame(src=output_file, width=1024, height=860)

✅ Map saved as results/example2.html


In [18]:
map_link = FileLink(path=output_file)
map_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/azure maps/results/example2.html

In [19]:
m = folium.Map(
    location=[test_response["results"][0]["position"]['lat'], test_response["results"][0]["position"]['lon']], 
    zoom_start=13,
    tiles='Esri.WorldImagery'
)

# Add alternative tile layers
folium.TileLayer('OpenStreetMap').add_to(m)
folium.TileLayer('Esri.WorldImagery').add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

folium.Marker(
    [test_response["results"][0]["position"]['lat'], test_response["results"][0]["position"]['lon']], 
    popup=address
).add_to(m)

output_file = os.path.join(RESULTS_DIR, "example3.html")
m.save(output_file)
print(f"✅ Map saved as {output_file}")

IFrame(src=output_file, width=1024, height=860)

✅ Map saved as results/example3.html


In [20]:
map_link = FileLink(path=output_file)
map_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/azure maps/results/example3.html

## Testing the routes features

In [21]:
def get_route_summary(loc1, loc2, travel_mode='car'):
    """
    Get route summary between two locations using Azure Maps API.
    
    Args:
        loc1 (list): Starting location as [latitude, longitude]
        loc2 (list): Destination location as [latitude, longitude]  
        travel_mode (str): Travel mode ('car', 'truck', 'taxi', 'bus', 'van', 'motorcycle', 'bicycle', 'pedestrian')
    
    Returns:
        dict: Route summary containing distance, travel time, etc.
    """
    routeUrl = 'https://atlas.microsoft.com/route/directions/json'
    query = f'{loc1[0]},{loc1[1]}:{loc2[0]},{loc2[1]}'
    
    params = {
        'api-version': 1.0,
        'subscription-key': subscription_key,
        'query': query,
        'travelMode': travel_mode,
        'traffic': True,
        'ComputeTravelTimeFor': 'all'
    }
    
    response = requests.get(routeUrl, params=params).json()
    return response['routes'][0]['summary']


def print_route_summary(summary):
    """
    Print route summary in a nice, readable format.
    
    Args:
        summary (dict): Route summary from Azure Maps API
    """
    print("🗺️  ROUTE SUMMARY")
    print("=" * 40)
    
    # Distance
    distance_km = summary.get('lengthInMeters', 0) / 1000
    print(f"📏 Distance: {distance_km:.1f} km")
    
    # Travel time without traffic
    travel_time_sec = summary.get('travelTimeInSeconds', 0)
    travel_time = str(timedelta(seconds=travel_time_sec))
    print(f"⏱️  Travel Time: {travel_time}")
    
    # Travel time with traffic (if available)
    if 'liveTrafficIncidentsTravelTimeInSeconds' in summary:
        traffic_time_sec = summary['liveTrafficIncidentsTravelTimeInSeconds']
        traffic_time = str(timedelta(seconds=traffic_time_sec))
        delay = traffic_time_sec - travel_time_sec
        print(f"🚦 With Traffic: {traffic_time}")
        if delay > 0:
            print(f"⚠️  Traffic Delay: +{delay//60} minutes")
    
    # Departure and arrival times (if available)
    if 'departureTime' in summary:
        print(f"🕐 Departure: {summary['departureTime']}")
    if 'arrivalTime' in summary:
        print(f"🏁 Arrival: {summary['arrivalTime']}")
    
    print("=" * 40)

def print_route_summary_json(summary):
    """Print route summary as formatted JSON."""
    print("📋 ROUTE SUMMARY (JSON)")
    print(json.dumps(summary, indent=5))

In [22]:
loc1 = [40.758896, -73.98513]
loc2 = [40.703564, -74.016678]


In [23]:
# Create a map centered between the two points
center_lat = (loc1[0] + loc2[0]) / 2
center_lon = (loc1[1] + loc2[1]) / 2

# Create the map
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add markers
folium.Marker(
    loc1,
    popup="Location 1",
    tooltip="Click for more info",
    icon=folium.Icon(color='red')
).add_to(m)

folium.Marker(
    loc2,
    popup="Location 2", 
    tooltip="Click for more info",
    icon=folium.Icon(color='blue')
).add_to(m)

# Optional: Add a line between the points
folium.PolyLine([loc1, loc2], color="green", weight=2.5, opacity=1).add_to(m)

# Save or display the map
m.save(os.path.join(RESULTS_DIR, 'map.html'))  # Saves as HTML file
m

In [32]:
map_link = FileLink(path=os.path.join(RESULTS_DIR, 'map.html'))
map_link

/mnt/batch/tasks/shared/LS_root/mounts/clusters/seretkow8/code/Users/seretkow/azure maps/results/map.html

In [24]:
summary = get_route_summary(loc1, loc2, 'car')
print_route_summary(summary)

🗺️  ROUTE SUMMARY
📏 Distance: 7.1 km
⏱️  Travel Time: 0:19:08
🚦 With Traffic: 0:19:08
🕐 Departure: 2025-09-02T04:40:12-04:00
🏁 Arrival: 2025-09-02T04:59:20-04:00


In [25]:
print_route_summary_json(summary)

📋 ROUTE SUMMARY (JSON)
{
     "lengthInMeters": 7139,
     "travelTimeInSeconds": 1148,
     "trafficDelayInSeconds": 0,
     "trafficLengthInMeters": 0,
     "departureTime": "2025-09-02T04:40:12-04:00",
     "arrivalTime": "2025-09-02T04:59:20-04:00",
     "noTrafficTravelTimeInSeconds": 1183,
     "historicTrafficTravelTimeInSeconds": 1198,
     "liveTrafficIncidentsTravelTimeInSeconds": 1148
}


In [26]:
summary = get_route_summary(loc1, loc2, 'truck')
print_route_summary(summary)

🗺️  ROUTE SUMMARY
📏 Distance: 8.5 km
⏱️  Travel Time: 0:21:39
🚦 With Traffic: 0:21:39
🕐 Departure: 2025-09-02T04:40:12-04:00
🏁 Arrival: 2025-09-02T05:01:51-04:00


In [27]:
summary = get_route_summary(loc1, loc2, 'bicycle')
print_route_summary(summary)

🗺️  ROUTE SUMMARY
📏 Distance: 7.1 km
⏱️  Travel Time: 0:23:01
🚦 With Traffic: 0:23:01
🕐 Departure: 2025-09-02T04:40:12-04:00
🏁 Arrival: 2025-09-02T05:03:13-04:00


In [28]:
summary = get_route_summary(loc1, loc2,'pedestrian')
print_route_summary(summary)

🗺️  ROUTE SUMMARY
📏 Distance: 6.8 km
⏱️  Travel Time: 1:22:05
🚦 With Traffic: 1:22:05
🕐 Departure: 2025-09-02T04:40:13-04:00
🏁 Arrival: 2025-09-02T06:02:18-04:00


In [29]:
summary = get_route_summary(loc1, loc2, 'taxi')
print_route_summary(summary)

🗺️  ROUTE SUMMARY
📏 Distance: 7.1 km
⏱️  Travel Time: 0:19:07
🚦 With Traffic: 0:19:07
🕐 Departure: 2025-09-02T04:40:13-04:00
🏁 Arrival: 2025-09-02T04:59:20-04:00


In [30]:
summary = get_route_summary(loc1, loc2, 'bus')
print_route_summary(summary)

🗺️  ROUTE SUMMARY
📏 Distance: 8.5 km
⏱️  Travel Time: 0:21:52
🚦 With Traffic: 0:21:52
🕐 Departure: 2025-09-02T04:40:13-04:00
🏁 Arrival: 2025-09-02T05:02:05-04:00
